# Object Detection scenario with RAI Dashboard

##### This notebook demonstrates how to

##### 1. Upload a private mltable dataset to AzureML
##### 2. Fine-tune an **AutoML Object Detection model (fasterrcnn_resnet50_fpn)** on the OD Fridge Dataset using an **individual run of an AzureML training job** and then
##### 3. Evaluate the AutoML model on the dataset using the RAI Vision Dashboard created using another AzureML pipeline.

##### Depending on your current Python environment, you may need to install the following packages

```
pip install azure-identity
pip install azure-ai-ml
pip install mlflow
pip install azureml-mlflow
```

In [ ]:
# !pip install -U azure-identity
# !pip install -U azure-ai-ml
# !pip install -U mlflow
# !pip install -U azureml-mlflow
# !pip install -U mltable

###### First, we need to specify the version of the RAI components which are available in the workspace. This was specified when the components were uploaded.

In [ ]:
version_string = "0.0.17"

###### We also need to give the name of the compute cluster we want to use in AzureML. Later in this notebook, we will create it if it does not already exist:

In [ ]:
# Compute cluster to run the AutoML training job
train_compute_name = "gpu-cluster-nc6-v3"

# Compute cluster to visualize and interact with the RAI Dashboard
rai_compute_name = "cpucluster"

# 1. Connect to Azure Machine Learning Workspace
###### The [workspace](https://learn.microsoft.com/en-us/azure/machine-learning/concept-workspace?view=azureml-api-2) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [ ]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

from azure.ai.ml.automl import SearchSpace, ObjectDetectionPrimaryMetrics
from azure.ai.ml import automl, dsl

In [ ]:
# Enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

## 1.2. Configure workspace details and get a handle to the workspace

###### To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](https://github.com/Azure/azureml-examples/blob/d8070da9dec29978e9322e727d1740f5486aa56e/sdk/python/jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
# Handle to the workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

try:
    credential = DefaultAzureCredential()
    ml_client = MLClient(
        credential=credential,
        subscription_id=subscription_id,
        resource_group_name=resource_group,
        workspace_name=workspace,
    )
except Exception:
    # If in compute instance we can get the config automatically
    from azureml.core import Workspace

    workspace = Workspace.from_config()
    workspace.write_config()
    ml_client = MLClient.from_config(
        credential=DefaultAzureCredential(exclude_shared_token_cache_credential=True),
        logging_enable=True,
    )

print(ml_client)

# 2. Compute target setup

###### You will need to provide a [Compute Target](https://docs.microsoft.com/en-us/azure/machine-learning/concept-azure-machine-learning-architecture#computes) that will be used for your AutoML model training. AutoML models for image tasks require [GPU SKUs](https://docs.microsoft.com/en-us/azure/virtual-machines/sizes-gpu) such as the ones from the NC, NCv2, NCv3, ND, NDv2 and NCasT4 series. We recommend using the NCsv3-series (with v100 GPUs) for faster training. Using a compute target with a multi-GPU VM SKU will leverage the multiple GPUs to speed up training. Additionally, setting up a compute target with multiple nodes will allow for faster model training by leveraging parallelism, when tuning hyperparameters for your model.

In [ ]:
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

try:
    _ = ml_client.compute.get(train_compute_name)
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    compute_config = AmlCompute(
        name=train_compute_name,
        type="amlcompute",
        size="Standard_NC6s_v3",
        idle_time_before_scale_down=120,
        min_instances=0,
        max_instances=4,
    )
    ml_client.begin_create_or_update(compute_config).result()

In [ ]:
from azure.ai.ml.entities import AmlCompute

all_compute_names = [x.name for x in ml_client.compute.list()]

if rai_compute_name in all_compute_names:
    print(f"Found existing compute: {rai_compute_name}")
else:
    rai_compute_config = AmlCompute(
        name=rai_compute_name,
        size="STANDARD_DS3_V2",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=3600,
    )
    ml_client.compute.begin_create_or_update(rai_compute_config)

# 3. MLTable with input Training Data

###### In order to generate models for computer vision tasks with automated machine learning, you need to bring labeled image data as input for model training in the form of an MLTable. You can create an MLTable from labeled training data in JSONL format. If your labeled training data is in a different format (like, pascal VOC or COCO), you can use a conversion script to first convert it to JSONL, and then create an MLTable. Alternatively, you can use Azure Machine Learning's [data labeling tool](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-image-labeling-projects) to manually label images, and export the labeled data to use for training your AutoML model.

###### In this notebook, we use a toy dataset called Fridge Objects, which consists of 128 images of 4 labels of beverage container {`can`, `carton`, `milk bottle`, `water bottle`} photos taken on different backgrounds.

###### All images in this notebook are hosted in [this repository](https://github.com/microsoft/computervision-recipes) and are made available under the [MIT license](https://github.com/microsoft/computervision-recipes/blob/master/LICENSE).

## 3.1. Download the Data
###### We first download and unzip the data locally. By default, the data would be downloaded in `./data` folder in current directory. 
###### If you prefer to download the data at a different location, update it in `dataset_parent_dir = ...` in the next cell.

In [ ]:
import os
import urllib
from zipfile import ZipFile

# Change to a different location if you prefer
dataset_parent_dir = "./data"

# create data folder if it doesnt exist.
os.makedirs(dataset_parent_dir, exist_ok=True)

# download data
download_url = "https://publictestdatasets.blob.core.windows.net/computervision/odFridgeObjects.zip"

# Extract current dataset name from dataset url
dataset_name = os.path.split(download_url)[-1].split(".")[0]
# Get dataset path for later use
dataset_dir = os.path.join(dataset_parent_dir, dataset_name)

# Get the data zip file path
data_file = os.path.join(dataset_parent_dir, f"{dataset_name}.zip")

# Download the dataset
urllib.request.urlretrieve(download_url, filename=data_file)

# extract files
with ZipFile(data_file, "r") as zip:
    print("extracting files...")
    zip.extractall(path=dataset_parent_dir)
    print("done")
# delete zip file
os.remove(data_file)

This is a sample image from this dataset:


In [ ]:
from IPython.display import Image

sample_image = os.path.join(dataset_dir, "images", "31.jpg")
Image(filename=sample_image)

## 3.2. Upload the images to Datastore through an AML Data asset (URI Folder)

###### In order to use the data for training in Azure ML, we upload it to our default Azure Blob Storage of our  Azure ML Workspace.

###### [Check this notebook for AML data asset example](https://github.com/Azure/azureml-examples/blob/d8070da9dec29978e9322e727d1740f5486aa56e/sdk/python/assets/data/data.ipynb)

In [ ]:
# Uploading image files by creating a 'data asset URI FOLDER':

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

my_data = Data(
    path=dataset_dir,
    type=AssetTypes.URI_FOLDER,
    description="Fridge-items images Object detection",
    name="fridge-items-images-object-detection",
)

uri_folder_data_asset = ml_client.data.create_or_update(my_data)

print(uri_folder_data_asset)
print("")
print("Path to folder in Blob Storage:")
print(uri_folder_data_asset.path)

## 3.3. Convert the downloaded data to JSONL

###### In this example, the fridge object dataset is annotated in Pascal VOC format, where each image corresponds to an xml file. Each xml file contains information on where its corresponding image file is located and also contains information about the bounding boxes and the object labels. 

###### For documentation on preparing the datasets beyond this notebook, please refer to the [documentation on how to prepare datasets](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-prepare-datasets-for-automl-images).


###### In order to use this data to create an AzureML MLTable, we first need to convert it to the required JSONL format. The following script is creating two `.jsonl` files (one for training and one for validation) in the corresponding MLTable folder. The train / validation ratio corresponds to 20% of the data going into the validation file. For further details on jsonl file used for image classification task in automated ml, please refer to the [data schema documentation for image object-detection task](https://learn.microsoft.com/en-us/azure/machine-learning/reference-automl-images-schema#object-detection).

### First generate JSONL files

###### The JSONL Conversion helpers require pycocotools and simplification packages

In [ ]:
%pip install pycocotools==2.0.6
%pip install simplification==0.6.11
%pip install scikit-image==0.19.3

In [ ]:
import sys

# use the jsonl-conversion files from automl examples folder
sys.path.insert(0, "../../jobs/automl-standalone-jobs/jsonl-conversion/")
from base_jsonl_converter import write_json_lines
from voc_jsonl_converter import VOCJSONLConverter

base_url = os.path.join(uri_folder_data_asset.path, "images/")
converter = VOCJSONLConverter(base_url, os.path.join(dataset_dir, "annotations"))
jsonl_annotations = os.path.join(dataset_dir, "annotations_voc.jsonl")
write_json_lines(converter, jsonl_annotations)

### Then split into train and validation

In [ ]:
import os

# We'll copy each JSONL file within its related MLTable folder
training_mltable_path = os.path.join(dataset_parent_dir, "training-mltable-folder")
validation_mltable_path = os.path.join(dataset_parent_dir, "validation-mltable-folder")

# First, let's create the folders if they don't exist
os.makedirs(training_mltable_path, exist_ok=True)
os.makedirs(validation_mltable_path, exist_ok=True)

train_validation_ratio = 5

# Path to the training and validation files
train_annotations_file = os.path.join(training_mltable_path, "train_annotations.jsonl")
validation_annotations_file = os.path.join(
    validation_mltable_path, "validation_annotations.jsonl"
)

with open(jsonl_annotations, "r") as annot_f:
    json_lines = annot_f.readlines()

index = 0
with open(train_annotations_file, "w") as train_f:
    with open(validation_annotations_file, "w") as validation_f:
        for json_line in json_lines:
            if index % train_validation_ratio == 0:
                # validation annotation
                validation_f.write(json_line)
            else:
                # train annotation
                train_f.write(json_line)
            index += 1

## 3.4. Convert annotation file from COCO to JSONL
###### If you want to try with a dataset in COCO format, see the example shown in Section 2 (specifically 2.4) of this [AutoML training notebook](https://github.com/Azure/azureml-examples/blob/main/sdk/python/jobs/automl-standalone-jobs/automl-image-object-detection-task-fridge-items/automl-image-object-detection-task-fridge-items.ipynb).

### Visualize bounding boxes
###### Please refer to the "Visualize data" section in the following [tutorial](https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-auto-train-image-models#visualize-data) to see how to easily visualize your ground truth bounding boxes before starting to train.

## 3.5. Create MLTable data input
###### Create MLTable data input using the jsonl files created above.

###### For documentation on creating your own MLTable assets for jobs beyond this notebook, please refer to below resources
###### - [MLTable YAML Schema](https://learn.microsoft.com/en-us/azure/machine-learning/reference-yaml-mltable) - covers how to write MLTable YAML, which is required for each MLTable asset.
###### - [Create MLTable data asset](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-data-assets?tabs=Python-SDK#create-a-mltable-data-asset) - covers how to create MLTable data asset. 

In [ ]:
def create_ml_table_file(filename):
    """Create ML Table definition"""

    return (
        "paths:\n"
        "  - file: ./{0}\n"
        "transformations:\n"
        "  - read_json_lines:\n"
        "        encoding: utf8\n"
        "        invalid_lines: error\n"
        "        include_path_column: false\n"
        "  - convert_column_types:\n"
        "      - columns: image_url\n"
        "        column_type: stream_info"
    ).format(filename)


def save_ml_table_file(output_path, mltable_file_contents):
    with open(os.path.join(output_path, "MLTable"), "w") as f:
        f.write(mltable_file_contents)


# Create and save train mltable
train_mltable_file_contents = create_ml_table_file(
    os.path.basename(train_annotations_file)
)
save_ml_table_file(training_mltable_path, train_mltable_file_contents)

# Save train and validation mltable
validation_mltable_file_contents = create_ml_table_file(
    os.path.basename(validation_annotations_file)
)
save_ml_table_file(validation_mltable_path, validation_mltable_file_contents)

In [ ]:
# Training MLTable defined locally, with local data to be uploaded
my_training_data_input = Input(type=AssetTypes.MLTABLE, path=training_mltable_path)

# Validation MLTable defined locally, with local data to be uploaded
my_validation_data_input = Input(type=AssetTypes.MLTABLE, path=validation_mltable_path)

# WITH REMOTE PATH: If available already in the cloud/workspace-blob-store
# my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/vision-classification/train")
# my_validation_data_input = Input(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/vision-classification/valid")

###### Specify the name of the label/target column containing the classes and bbox information. (NOTE: This variable is used later in the RAI Dashboard section of the notebook as well.)

In [ ]:
target_column_name = "label"

# 4. Configure and run the AutoML for Images Object Detection training job

###### AutoML allows you to easily train models for Image Classification, Object Detection & Instance Segmentation on your image data. You can control the model algorithm and hyperparameters to be used, perform a sweep over a manually specified hyperparameter space, or the system can automatically perform a hyperparameter sweep for you.

###### To explore different training options with AutoML, have a look at Section of 4 of this [example notebook that demonstrates how to train AutoML models for Object Detection.](https://github.com/Azure/azureml-examples/blob/main/sdk/python/jobs/automl-standalone-jobs/automl-image-object-detection-task-fridge-items/automl-image-object-detection-task-fridge-items.ipynb)


## Individual AutoML training run

###### You can launch individual runs to explore model algorithms; we provide sensible default hyperparameters for each algorithm. You can also launch individual runs for the same model algorithm and different hyperparameter combinations. The model algorithm is specified using the model_name parameter. Please refer to the [documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-image-models?tabs=CLI-v2#configure-model-algorithms-and-hyperparameters) for the list of supported model algorithms.

###### The following function can be used to configure AutoML jobs for individual runs:
### set_training_parameters() function parameters:
###### This is an optional configuration method to configure fixed settings or parameters that don't change during the parameter space sweep. Some of the key parameters of this function are:

###### - `model_name` - The name of the ML algorithm that we want to use in training job. Please refer to this [documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-image-models?tabs=CLI-v2#supported-model-algorithms) for supported model algorithm.
###### - `number_of_epochs` - The number of training epochs. It must be positive integer (default value is 15).
###### - `layers_to_freeze` - The number of layers to freeze in model for transfer learning. It must be a positive integer (default value is 0).
###### - `early_stopping` - It enable early stopping logic during training, It must be boolean value (default is True).   
###### - `optimizer` - Type of optimizer to use in training. It must be either sgd, adam, adamw (default is sgd).
###### - `distributed` - It enable distributed training if compute target contain multiple GPUs. It must be boolean value (default is True).

###### If you wish to use the default hyperparameter values for a given algorithm (say `yolov5`), you can specify the job for your AutoML Image runs as follows:

In [ ]:
# general job parameters
exp_name = "dpv2-odfridge-automl-training"

In [ ]:
# Create the AutoML job with the related factory-function.
image_object_detection_job = automl.image_object_detection(
    compute=train_compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    validation_data=my_validation_data_input,
    target_column_name=target_column_name,
    primary_metric=ObjectDetectionPrimaryMetrics.MEAN_AVERAGE_PRECISION,
    tags={"data": "ODFridge", "model type": "AutoML"},
)

# Set limits
image_object_detection_job.set_limits(timeout_minutes=60)

# Pass the fixed settings or parameters
image_object_detection_job.set_training_parameters(
    model_name="fasterrcnn_resnet50_fpn",
    early_stopping=1,
    number_of_epochs=1,
    learning_rate=0.09,
)

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(image_object_detection_job)

print(f"Created job: {returned_job}")

In [ ]:
ml_client.jobs.stream(returned_job.name)

# 5. Retrieve the Best Trial (Best Model's trial/run) and Register best model
###### Use the MLFLowClient to access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Trial.

## 5.1 Initialize MLFlow Client and Obtain the tracking URI for MLFlow

###### The models and artifacts that are produced by AutoML can be accessed via the MLFlow interface.
###### Initialize the MLFlow client here, and set the backend as Azure ML, via. the MLFlow Client.

###### IMPORTANT, you need to have installed the latest MLFlow packages with:

    pip install azureml-mlflow

    pip install mlflow

In [ ]:
import mlflow

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

In [ ]:
# Set the MLFLOW TRACKING URI
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
print(f"\nCurrent tracking uri: {mlflow.get_tracking_uri()}")

In [ ]:
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

### 5.2 Get the AutoML parent Job and Get the AutoML best child run

In [ ]:
job_name = returned_job.name

# # Example if providing an specific Job name/ID
# job_name = "happy_yam_40fq53m7c2" #"ashy_net_gdd31zf2fq"

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)

print("Parent Run: ")
print(mlflow_parent_run)

In [ ]:
# Print parent run tags. 'automl_best_child_run_id' tag should be there.
print(mlflow_parent_run.data.tags.keys())

In [ ]:
# Get the best model's child run

best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print(f"Found best child run id: {best_child_run_id}")

best_run = mlflow_client.get_run(best_child_run_id)

print("Best child run: ")
print(best_run)

In [ ]:
import pandas as pd

# Access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Run.
pd.DataFrame(best_run.data.metrics, index=[0]).T

## 5.3 Download the best model locally
###### Access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Run.

In [ ]:
# Create local folder
import os

local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [ ]:
# Download run's artifacts/outputs
local_path = mlflow_client.download_artifacts(
    best_run.info.run_id, "outputs", local_dir
)
print(f"Artifacts downloaded in: {local_path}")
print(f"Artifacts: {os.listdir(local_path)}")

In [ ]:
import os

mlflow_model_dir = os.path.join(local_dir, "outputs", "mlflow-model")

# Show the contents of the MLFlow model folder
os.listdir(mlflow_model_dir)

# You should see a list of files such as the following:
# ['artifacts', 'conda.yaml', 'MLmodel', 'python_env.yaml', 'python_model.pkl', 'requirements.txt']

## 5.4 Register Model

In [ ]:
# import required libraries
from azure.ai.ml.entities import Model

model_name = "automl-fasterrcnn-odfridge-model"

model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/mlflow-model/",
    name=model_name,
    description="AutoML FasterRCNN model trained on OD Fridge Dataset",
    type=AssetTypes.MLFLOW_MODEL,
)

# for downloaded file
# model = Model(
#     path=mlflow_model_dir,
#     name=model_name,
#     description="AutoML FasterRCNN model trained on OD Fridge Dataset",
#     type=AssetTypes.MLFLOW_MODEL,
# )

registered_model = ml_client.models.create_or_update(model)

In [ ]:
registered_model.id

# 5.5 (Optional) If you already have a registered AutoML Object Detection model

In [ ]:
# model_name = "automl-fasterrcnn-odfridge-model"
# version_number = 4

# registered_model = ml_client.models.get(model_name, version=version_number)
# print(registered_model.name, registered_model.version)

###### Specify the name of the label/target column containing the classes and bbox information. (NOTE: This variable is used later in the RAI Dashboard section of the notebook as well.)

In [ ]:
# target_column_name = "label"

# 6. Creating the RAI Vision Insights

###### Now that we have our model, we can generate RAI Vision insights for it. We will need the `id` of the registered model, which will be as follows:

In [ ]:
expected_model_id = f"{registered_model.name}:{registered_model.version}"
azureml_model_id = f"azureml:{expected_model_id}"

## 6.1 Prepare the data you want to analyze with the RAI Dashboard
#### NOTE: This example notebook demonstrates how to use a private OD Fridge Dataset in AzureML

In [ ]:
try:
    from urllib import urlretrieve
except ImportError:
    from urllib.request import urlretrieve

# from enum import Enum
import xml.etree.ElementTree as ET
import pandas as pd
from zipfile import ZipFile

from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

In [ ]:
fridge_test_mltable = Input(type=AssetTypes.MLTABLE, path=validation_mltable_path)

## 6.3 Load the RAI Components
###### Next, we load the RAI components, so that we can construct a pipeline:

In [ ]:
registry_name = "azureml"
credential = DefaultAzureCredential()

ml_client_registry = MLClient(
    credential=credential,
    subscription_id=ml_client.subscription_id,
    resource_group_name=ml_client.resource_group_name,
    registry_name=registry_name,
)

rai_vision_insights_component = ml_client_registry.components.get(
    name="rai_vision_insights", version=version_string
)

## 6.4 Constructing the pipeline in sdk
###### We can now specify our pipeline. Complex objects (such as lists of column names) have to be converted to JSON strings before being passed to the components.

In [ ]:
import json
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes


@dsl.pipeline(
    compute=rai_compute_name,
    description="Example RAI computation on Fridge data with AutoML FasterRCNN Object Detection model",
    experiment_name=f"RAI_Fridge_Example_RAIInsights_Computation_{expected_model_id}",
)
def rai_fridge_object_detection_pipeline(target_column_name, test_data, classes):
    # Initiate the RAIInsights
    rai_image_job = rai_vision_insights_component(
        task_type="object_detection",
        model_info=expected_model_id,
        model_input=Input(type=AssetTypes.MLFLOW_MODEL, path=azureml_model_id),
        test_dataset=test_data,
        target_column_name=target_column_name,
        classes=classes,
        model_type="pyfunc",
        precompute_explanation=True,
        dataset_type="private",
        enable_error_analysis=False,
        maximum_rows_for_test_dataset=5000,
        num_masks=300,
        mask_res=4,
    )
    rai_image_job.set_limits(timeout=24000)

    rai_image_job.outputs.dashboard.mode = "upload"
    rai_image_job.outputs.ux_json.mode = "upload"

    return {
        "dashboard": rai_image_job.outputs.dashboard,
        "ux_json": rai_image_job.outputs.ux_json,
    }

###### Next, we define the pipeline object itself, and ensure that the outputs will be available for download:

In [ ]:
import uuid
from azure.ai.ml import Output

insights_pipeline_job = rai_fridge_object_detection_pipeline(
    target_column_name=target_column_name,
    test_data=fridge_test_mltable,
    classes='["can", "carton", "milk_bottle", "water_bottle"]',  # Ensure the class order matches that of used while creating the jsonl file for the test data
)

rand_path = str(uuid.uuid4())
insights_pipeline_job.outputs.dashboard = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/dashboard/",
    mode="upload",
    type="uri_folder",
)
insights_pipeline_job.outputs.ux_json = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/ux_json/",
    mode="upload",
    type="uri_folder",
)

###### And submit the pipeline to AzureML for execution:

In [ ]:
# Submit the RAI Vision Insights job
returned_job = ml_client.jobs.create_or_update(insights_pipeline_job)

print(f"Created job: {returned_job}")

In [ ]:
ml_client.jobs.stream(returned_job.name)

###### The dashboard should appear in the AzureML portal in the registered model view. The following cell computes the expected URI:

In [ ]:
sub_id = ml_client._operation_scope.subscription_id
rg_name = ml_client._operation_scope.resource_group_name
ws_name = ml_client.workspace_name

expected_uri = f"https://ml.azure.com/model/{expected_model_id}/model_analysis?wsid=/subscriptions/{sub_id}/resourcegroups/{rg_name}/workspaces/{ws_name}"

print(f"Please visit {expected_uri} to see your analysis")